# Telecom X – Parte 2: Predicción de Cancelación (Churn)

## 📣 Historia del Desafío

¡Felicidades! 🎉 Has sido promovido después de tu excelente desempeño en el análisis exploratorio de la cancelación de clientes en Telecom X. Tu dedicación, claridad al comunicar los datos y visión estratégica marcaron la diferencia.

Ahora, ¡has sido invitado oficialmente a formar parte del equipo de Machine Learning de la empresa!

## 🎯 Misión

Tu nueva misión es desarrollar modelos predictivos capaces de prever qué clientes tienen mayor probabilidad de cancelar sus servicios.

La empresa quiere anticiparse al problema de la cancelación, y te corresponde a ti construir un pipeline robusto para esta etapa inicial de modelado.

## 🧠 Objetivos del Desafío

Preparar los datos para el modelado (tratamiento, codificación, normalización).

Realizar análisis de correlación y selección de variables.

Entrenar dos o más modelos de clasificación.

Evaluar el rendimiento de los modelos con métricas.

Interpretar los resultados, incluyendo la importancia de las variables.

Crear una conclusión estratégica señalando los principales factores que influyen en la cancelación.

## 🧰 Lo que vas a practicar

✅ Preprocesamiento de datos para Machine Learning
✅ Construcción y evaluación de modelos predictivos
✅ Interpretación de resultados y entrega de insights
✅ Comunicación técnica con enfoque estratégico

## 🚀 Ahora eres: Analista Junior de Machine Learning

Telecom X confía en tu entrega para dar los próximos pasos hacia una solución de inteligencia predictiva eficaz. ¡Buena suerte!

# Carga

Comenzaremos por cargar el archivo previamente modificado en el desafío anterior

In [2]:
import pandas as pd

In [12]:
datos = pd.read_csv("telecomx_data_processed.csv")
datos.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Monthly,Total,Cuentas_Diarias,Cantidad_Servicios,MonthlyGroup
0,0002-ORFBO,0,Female,0,1,1,9,1,0,DSL,...,1,0,One year,1,Mailed check,65.6,593.30,2.186667,4,Medio
1,0003-MKNFE,0,Male,0,0,0,9,1,1,DSL,...,0,1,Month-to-month,0,Mailed check,59.9,542.40,1.996667,3,Medio
2,0004-TLHLJ,1,Male,0,0,0,4,1,0,Fiber optic,...,0,0,Month-to-month,1,Electronic check,73.9,280.85,2.463333,2,Alto
3,0011-IGKFF,1,Male,1,1,0,13,1,0,Fiber optic,...,1,1,Month-to-month,1,Electronic check,98.0,1237.85,3.266667,5,Alto
4,0013-EXCHZ,1,Female,1,1,0,3,1,0,Fiber optic,...,1,0,Month-to-month,1,Mailed check,83.9,267.40,2.796667,3,Alto


In [13]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   customerID          7043 non-null   object 
 1   Churn               7043 non-null   int64  
 2   gender              7043 non-null   object 
 3   SeniorCitizen       7043 non-null   int64  
 4   Partner             7043 non-null   int64  
 5   Dependents          7043 non-null   int64  
 6   tenure              7043 non-null   int64  
 7   PhoneService        7043 non-null   int64  
 8   MultipleLines       7043 non-null   int64  
 9   InternetService     7043 non-null   object 
 10  OnlineSecurity      7043 non-null   int64  
 11  OnlineBackup        7043 non-null   int64  
 12  DeviceProtection    7043 non-null   int64  
 13  TechSupport         7043 non-null   int64  
 14  StreamingTV         7043 non-null   int64  
 15  StreamingMovies     7043 non-null   int64  
 16  Contra

Vamos a eliminar la columna de "customerID" ya que no es relevante para el modelo

In [14]:
datos_1 = datos.drop("customerID", axis=1)

In [18]:
for col in datos_1.columns:
    print(f"Columna: {col}")
    print(datos_1[col].unique())

Columna: Churn
[0 1]
Columna: gender
['Female' 'Male']
Columna: SeniorCitizen
[0 1]
Columna: Partner
[1 0]
Columna: Dependents
[1 0]
Columna: tenure
[ 9  4 13  3 71 63  7 65 54 72  5 56 34  1 45 50 23 55 26 69 37 49 66 67
 20 43 59 12 27  2 25 29 14 35 64 39 40 11  6 30 70 57 58 16 32 33 10 21
 61 15 44 22 24 19 47 62 46 52  8 60 48 28 41 53 68 31 36 17 18 51 38 42
  0]
Columna: PhoneService
[1 0]
Columna: MultipleLines
[0 1]
Columna: InternetService
['DSL' 'Fiber optic' 'No']
Columna: OnlineSecurity
[0 1]
Columna: OnlineBackup
[1 0]
Columna: DeviceProtection
[0 1]
Columna: TechSupport
[1 0]
Columna: StreamingTV
[1 0]
Columna: StreamingMovies
[0 1]
Columna: Contract
['One year' 'Month-to-month' 'Two year']
Columna: PaperlessBilling
[1 0]
Columna: PaymentMethod
['Mailed check' 'Electronic check' 'Credit card (automatic)'
 'Bank transfer (automatic)']
Columna: Monthly
[65.6  59.9  73.9  ... 91.75 68.8  67.85]
Columna: Total
[ 593.3   542.4   280.85 ...  742.9  4627.65 3707.6 ]
Columna: C

Ahora sabemos que elementos debemos codificar para hacer funcionar mejor nuestro modelo

In [19]:
datos_1['gender'] = datos_1['gender'].map({'Female': 0, 'Male': 1})

In [23]:
columnas_categorticas = ['InternetService', 'Contract', 'PaymentMethod', 'MonthlyGroup']
datos_encoded = pd.get_dummies(datos_1, columns=columnas_categorticas, drop_first=True, dtype=int)
datos_encoded.head()

,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,...,InternetService_Fiber optic,InternetService_No,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,MonthlyGroup_Bajo,MonthlyGroup_Medio,MonthlyGroup_Muy Alto
0,0,0,0,1,1,9,1,0,0,1,...,0,0,1,0,0,0,1,0,1,0
1,0,1,0,0,0,9,1,1,0,0,...,0,0,0,0,0,0,1,0,1,0
2,1,1,0,0,0,4,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0
3,1,1,1,1,0,13,1,0,0,1,...,1,0,0,0,0,1,0,0,0,0
4,1,0,1,1,0,3,1,0,0,0,...,1,0,0,0,0,0,1,0,0,0


## Verificación de la Proporción de Cancelación (Churn)

In [25]:
# Proporciones en porcentaje
datos_encoded['Churn'].value_counts(normalize=True) * 100

Churn
0    73.463013
1    26.536987
Name: proportion, dtype: float64

Podemos observar que los clientes que no cancelaron representan el 73.46% del total. Esto trae un desbalance en el modelo ya que lo que más queremos predecir es que clientes van a abandonar.
Si no solucionamos este desbalance, nuestro modelo será muy bueno en ver que cliente no abandona, pero perderá el foco de encontrar que cliente puede llegar a abandonar.

In [26]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

In [27]:
X = datos_encoded.drop('Churn', axis=1)
y = datos_encoded['Churn']

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=11)

Vamos a aplicar SMOTE en el set de entrenamiento y dejar sin cambios el set de pruebas

In [30]:
print(f"Antes de realizar SMOTE, tenemos {y_train.value_counts(normalize=True) * 100}")

Antes de realizar SMOTE, tenemos Churn
0    73.464679
1    26.535321
Name: proportion, dtype: float64


In [31]:
smote = SMOTE(random_state=11)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
print(f"Después de realizar SMOTE, tenemos {y_train_resampled.value_counts(normalize=True) * 100}")

Después de realizar SMOTE, tenemos Churn
1    50.0
0    50.0
Name: proportion, dtype: float64


## Normalización de los datos

In [32]:
from sklearn.preprocessing import StandardScaler

In [33]:
cols_to_scale = ['tenure', 'Monthly', 'Total', 'Cuentas_Diarias', 'Cantidad_Servicios']
scaler = StandardScaler()

# Entrenamiento
X_train_resampled[cols_to_scale] = scaler.fit_transform(X_train_resampled[cols_to_scale])

# Prueba
X_test[cols_to_scale] = scaler.transform(X_test[cols_to_scale])